In [2]:
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.linear_model import RANSACRegressor
from scipy.spatial import ConvexHull
import logging
from scipy.spatial.distance import pdist, squareform
import open3d as o3d
import numpy as np
from sklearn.linear_model import RANSACRegressor
from scipy.spatial import cKDTree
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist

# Load the segmented point cloud
pcd = o3d.io.read_point_cloud("/home/t/Desktop/work/results/combined_pointcloud1.pcd")

# Visualize the point cloud to check its structure
o3d.visualization.draw_geometries([pcd])

# Print basic information about the point cloud
print(f"Points: {len(pcd.points)}")
print(f"Has normals: {pcd.has_normals()}")
print(f"Has colors: {pcd.has_colors()}")

Points: 22561
Has normals: True
Has colors: True


In [7]:
import open3d as o3d
import numpy as np
from sklearn.linear_model import RANSACRegressor
from scipy.spatial import cKDTree
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

# Load the segmented point cloud
pcd = o3d.io.read_point_cloud("/home/t/Desktop/work/results/combined_pointcloud1.pcd")

# Function to separate the point cloud into individual planes based on color
def separate_planes_by_color(pcd):
    colors = np.asarray(pcd.colors)
    unique_colors = np.unique(colors, axis=0)
    planes = []
    for color in unique_colors:
        color_mask = np.all(colors == color, axis=1)
        plane = pcd.select_by_index(np.where(color_mask)[0])
        planes.append(plane)
    return planes

# Separate the point cloud into individual planes
planes = separate_planes_by_color(pcd)

# Function to fit a plane to a set of points using RANSAC
def fit_plane_ransac(points):
    ransac = RANSACRegressor()
    X = points[:, :2]  # Use x, y for fitting
    y = points[:, 2]   # Use z as the target
    ransac.fit(X, y)
    normal = np.array([-ransac.estimator_.coef_[0], -ransac.estimator_.coef_[1], 1])
    normal /= np.linalg.norm(normal)
    centroid = np.mean(points, axis=0)
    return normal, centroid

# Function to find points close to both planes
def find_close_points(points1, points2, normal1, normal2, threshold=0.08):
    tree1 = cKDTree(points1)
    tree2 = cKDTree(points2)
    
    # Add angle threshold between normals (e.g., minimum 30 degrees)
    angle = np.arccos(np.clip(np.dot(normal1, normal2), -1.0, 1.0))
    if angle < np.pi/6:  # Skip if planes are nearly parallel
        return np.array([])
    
    close_points = []
    for point in points1:
        dist, _ = tree2.query(point)
        if dist < threshold:
            # Check if point is close to both planes
            dist1 = np.abs(np.dot(point - points1[0], normal1))
            dist2 = np.abs(np.dot(point - points2[0], normal2))
            if dist1 < threshold and dist2 < threshold:
                close_points.append(point)
    
    return np.array(close_points)

# Function to fit a line to a set of points
def fit_line_to_points(points):
    if len(points) < 2:
        return None, None
    
    # Add minimum points threshold
    if len(points) < 10:  # Adjust this threshold based on your point cloud density
        return None, None
    
    pca = PCA(n_components=1)
    pca.fit(points)
    
    # Add explained variance ratio threshold
    if pca.explained_variance_ratio_[0] < 0.8:  # Points should be roughly linear
        return None, None
    
    direction = pca.components_[0]
    centroid = np.mean(points, axis=0)
    
    # Project points onto the line
    projected_points = np.dot(points - centroid, direction[:, np.newaxis]) * direction + centroid
    
    # Find the two extreme points
    distances = np.dot(projected_points - centroid, direction)
    min_idx, max_idx = np.argmin(distances), np.argmax(distances)
    
    # Add minimum length threshold
    edge_length = np.linalg.norm(projected_points[max_idx] - projected_points[min_idx])
    if edge_length < 0.5:  # Adjust this threshold based on your scene scale
        return None, None
    
    return projected_points[min_idx], projected_points[max_idx]

# Find potential diffraction edges
def find_diffraction_edges(planes):
    diffraction_edges = []
    for i in range(len(planes)):
        for j in range(i + 1, len(planes)):
            points1 = np.asarray(planes[i].points)
            points2 = np.asarray(planes[j].points)
            
            normal1, _ = fit_plane_ransac(points1)
            normal2, _ = fit_plane_ransac(points2)
            
            close_points = find_close_points(points1, points2, normal1, normal2)
            if len(close_points) > 1:
                start_point, end_point = fit_line_to_points(close_points)
                if start_point is not None and end_point is not None:
                    diffraction_edges.append((start_point, end_point))
    
    return diffraction_edges

# Find potential diffraction edges
diffraction_edges = find_diffraction_edges(planes)

# Visualize diffraction edges
def visualize_diffraction_edges(diffraction_edges):
    edge_lines = []
    for start_point, end_point in diffraction_edges:
        line_set = o3d.geometry.LineSet()
        line_set.points = o3d.utility.Vector3dVector([start_point, end_point])
        line_set.lines = o3d.utility.Vector2iVector([[0, 1]])
        line_set.paint_uniform_color([1, 0, 0])  # Red color for diffraction edges
        edge_lines.append(line_set)
    
    o3d.visualization.draw_geometries(edge_lines + [pcd])

if diffraction_edges:
    visualize_diffraction_edges(diffraction_edges)
else:
    print("No diffraction edges found.")